# MLP Image Classifier
# Train

Let's take the following steps:

1. Encoding target variable
2. Training the KNN model
3. Export the model


## Step 01: Setup

Start out by installing the experiment tracking library and setting up your free W&B account:


*   **pip install wandb** – Install the W&B library
*   **import wandb** – Import the wandb library
*   **wandb login** – Login to your W&B account so you can log all your metrics in one place

In [ ]:
!pip install wandb -qU

In [ ]:
import wandb
wandb.login()

### Import Packages

In [ ]:
# import the necessary packages
from imutils import paths
import logging
import os
import cv2
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score

In [ ]:
# configure logging
# reference for a logging obj
logger = logging.getLogger()

# set level of logging
logger.setLevel(logging.INFO)

# create handlers
c_handler = logging.StreamHandler()
c_format = logging.Formatter(fmt="%(asctime)s %(message)s",datefmt='%d-%m-%Y %H:%M:%S')
c_handler.setFormatter(c_format)

# add handler to the logger
logger.handlers[0] = c_handler

## Step 02: Train

In [ ]:
# since we are using Jupyter Notebooks we can replace our argument
# parsing code with *hard coded* arguments and values
args = {
  "project_name": "mlp_image_classifier",
  "train_feature_artifact": "train_x:latest",
  "train_target_artifact": "train_y:latest",
  "neighbors": 1,
  "jobs": -1,
  "encoder": "target_encoder",
  "inference_model": "model"
}

In [ ]:
# open the W&B project created in the Fetch step
run = wandb.init(entity="marianabritoazevedo",project=args["project_name"], job_type="Train")

logger.info("Downloading the training data")
train_x_artifact = run.use_artifact(args["train_feature_artifact"])
train_x_path = train_x_artifact.file()
train_y_artifact = run.use_artifact(args["train_target_artifact"])
train_y_path = train_y_artifact.file()

# unpacking the artifacts
train_x = joblib.load(train_x_path)
train_y = joblib.load(train_y_path)

In [ ]:
# encode the labels as integers
le = LabelEncoder()
train_y = le.fit_transform(train_y)

# train a k-NN classifier on the raw pixel intensities
logger.info("[INFO] training MLP classifier...")
model = MLPClassifier(hidden_layer_sizes=(128, 128), activation='relu', solver='adam')
model.fit(train_x, train_y)

In [ ]:
logger.info("Dumping the model and encoder artifacts to the disk")

# Save the artifacts using joblib
joblib.dump(le, args["encoder"])
joblib.dump(model, args["inference_model"])

In [ ]:
# encoder artifact
artifact = wandb.Artifact(args["encoder"],
                          type="INFERENCE_MODEL",
                          description="A json file representing the target encoder"
                          )

logger.info("Logging the target encoder artifact")
artifact.add_file(args["encoder"])
run.log_artifact(artifact)

In [ ]:
# inference model artifact
artifact = wandb.Artifact(args["inference_model"],
                          type="INFERENCE_MODEL",
                          description="A json file representing the inference model"
                          )

logger.info("Logging the inference model artifact")
artifact.add_file(args["inference_model"])
run.log_artifact(artifact)

In [ ]:
run.finish()

## Step 03: Hyperparameter tuning with sweep

In [ ]:
sweep_config = {
    "name": "mlp-sweep",
    "metric": {"name": "accuracy", "goal": "maximize"},
    "method": "grid",
    "parameters": {
        "hidden_layer_sizes": {
            "values":[(128,128),(128,64),(128,128,64),(128, 64, 32, 16)]
        },
        "activation": {
            "values": ['relu', 'tanh']
        },
        "solver": {
            "values": ['adam', 'sgd']
        },
        "learning_rate" : {
            "values": ['constant','adaptive']
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project=args['project_name'])

In [ ]:
# partition the data into training and validation splits using 75% of
# the data for training and the remaining 25% for validation
(train_x, val_x, train_y, val_y) = train_test_split(train_x, train_y,test_size=0.25, random_state=42)

In [ ]:
def train():
  with wandb.init() as run:

    model = MLPClassifier(hidden_layer_sizes=run.config.hidden_layer_sizes,
                          activation=run.config.activation,
                          solver=run.config.solver,
                          learning_rate=run.config.learning_rate)

    # training
    logger.info("Training")
    model.fit(train_x,train_y)

    # infering
    logger.info("Infering")
    predict = model.predict(val_x)

    # Evaluation Metrics
    logger.info("Test Evaluation metrics")
    fbeta = fbeta_score(val_y, predict, beta=1, zero_division=1,average='weighted')
    precision = precision_score(val_y, predict, zero_division=1,average='weighted')
    recall = recall_score(val_y, predict, zero_division=1,average='weighted')
    acc = accuracy_score(val_y, predict)

    logger.info("Test Accuracy: {}".format(acc))
    logger.info("Test Precision: {}".format(precision))
    logger.info("Test Recall: {}".format(recall))
    logger.info("Test F1: {}".format(fbeta))

    run.summary["Acc"] = acc
    run.summary["Precision"] = precision
    run.summary["Recall"] = recall
    run.summary["F1"] = fbeta

    run.finish()

In [ ]:
wandb.agent(sweep_id, function=train)

In [ ]:
run.finish()